Benchmarking RMSD analysis speed using cloud & disk reading

Prerequisites:
- `benchmark_setup.ipynb`
- `align_write_benchmark.ipynb`

Steps:

1. Open a `zarr.Group` object from the aligned trajectory stored on disk
2. Open a group from the trajectory uploaded to an AWS S3 bucket
3. Create an `mda.Universe` object for both zarr groups and one for the original .xtc trajectory
4. Perform the RMSD analysis for each `mda.Universe`, time, and record results

In [ ]:
import zarrtraj
import MDAnalysis as mda
import zarr
from zarr.storage import LRUStoreCache
import s3fs
import os

# 1
yiipHDD = zarr.open_group("notebook_data_tmp/yiip_aligned.zarrtraj")

# 2
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
BUCKET_NAME = os.getenv("BUCKET_NAME")

s3 = s3fs.S3FileSystem(key=AWS_ACCESS_KEY_ID, secret=AWS_SECRET_ACCESS_KEY)
store = s3fs.S3Map(root='zarrtraj-test-data/yiip_aligned.zarrtraj', s3=s3, check=False)
# Select max_size value in bytes based on chunking of zarrtraj data
# At least one chunk must fit in the cache
cache = LRUStoreCache(store, max_size=2**25)
yiipS3 = zarr.open_group(store=cache)

# 3
uHDD = mda.Universe("notebook_data_tmp/YiiP_system.pdb", yiipHDD)
uS3 = mda.Universe("notebook_data_tmp/YiiP_system.pdb", yiipS3)
uXTC = mda.Universe("notebook_data_tmp/YiiP_system.pdb", "YiiP_system_90ns_center.xtc")

In [ ]:
from MDAnalysis.analysis import rms
import time

universes = dict()
universes["uHDD"]["ref"] = uHDD
universes["uS3"]["ref"] = uS3
universes["uXTC"]["ref"] = uXTC


for name in ("uHDD", "uS3", "uXTC"):
    start = time.time()
    R = rms.RMSD(universes[name]["ref"],
                universes[name]["ref"],
                select='backbone',
                ref_frame=0)
    stop = time.time()
    universes[name]["RMSD"] = stop - start

    